## Data origin :: entsoe

The origin for this data is the transparency platform "entsoe". It's a central collection and publiaction of electricity generation, transportation and consumption data and information for the pan-European market. The url for the following data is: https://transparency.entsoe.eu/balancing/r2/imbalance/show


### Original features

The instances represent article orders. The given features are as follows:

## Part 1: Data mining

We import the data set, explore it briefly, drop duplicates and unused features and cast the data types.

## Imports

### Modules, classes and functions

In [614]:
import datetime, time, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import json

from cesium import datasets

from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from datetime import datetime, timezone, timedelta

resample_size = "15MIN"
resample_factor = 15

#windowfactor = 15

import warnings

In [615]:
def cleaningcolumns(df):
    
    df.rename(str.lower, axis='columns', inplace=True)
    df.columns = df.columns.str.replace('(', '')
    df.columns = df.columns.str.replace(')', '')
    df.columns = df.columns.str.replace(' ','')
    df.columns = df.columns.str.replace('-','')
    df.columns = df.columns.str.replace('|','')
    df.columns = df.columns.str.replace(']','')
    df.columns = df.columns.str.replace('[','')
    df.columns = df.columns.str.replace('+','plus')
    df.columns = df.columns.str.replace('-','minus')
    df.columns = df.columns.str.replace('/','')

## Import Previous Data 

In [616]:
df_merged_1 = pd.read_pickle('data/pickle/df_merged_1.pickle')

## Data

We import our data, which is available in a single csv file, in a dataframe.

In [617]:
df_imbalance_de = pd.read_csv("data/entsoe/df_imbalance_de.csv")

new = df_imbalance_de["Imbalance settlement period (UTC)"].str.split(" - ", n=2, expand=True)

df_imbalance_de["dt_start_utc"]= new[0]
df_imbalance_de["dt_start_utc"] = pd.to_datetime(df_imbalance_de["dt_start_utc"], dayfirst=True)
df_imbalance_de.set_index('dt_start_utc', inplace=True)

df_imbalance_de.columns = df_imbalance_de.columns.str.replace(' ','')
df_imbalance_de.columns = df_imbalance_de.columns.str.replace('-','')
df_imbalance_de.columns = df_imbalance_de.columns.str.replace('|','')
df_imbalance_de.columns = df_imbalance_de.columns.str.replace(']','')
df_imbalance_de.columns = df_imbalance_de.columns.str.replace('[','')
df_imbalance_de.columns = df_imbalance_de.columns.str.replace('+','plus')
df_imbalance_de.columns = df_imbalance_de.columns.str.replace('-','minus')
df_imbalance_de.columns = df_imbalance_de.columns.str.replace('/','')

/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2980905474.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_imbalance_de.columns = df_imbalance_de.columns.str.replace('|','')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2980905474.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_imbalance_de.columns = df_imbalance_de.columns.str.replace(']','')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2980905474.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_imbal

In [618]:
df_imbalance_de.head()

,Imbalancesettlementperiod(UTC),plusImbalancePriceEURMWhMBADELU,ImbalancePriceEURMWhMBADELU,Status,TotalImbalanceMWhMBADELU,Difference,Situation,Status.1
dt_start_utc,,,,,,,,
2021-01-01 00:00:00,01.01.2021 00:00 - 01.01.2021 00:15,96.09,96.09,Final,52,NaN,Deficit,Intermediate
2021-01-01 00:15:00,01.01.2021 00:15 - 01.01.2021 00:30,94.62,94.62,Final,7,NaN,Deficit,Intermediate
2021-01-01 00:30:00,01.01.2021 00:30 - 01.01.2021 00:45,92.21,92.21,Final,13,NaN,Deficit,Intermediate
2021-01-01 00:45:00,01.01.2021 00:45 - 01.01.2021 01:00,27.64,27.64,Final,38,NaN,Surplus,Intermediate
2021-01-01 01:00:00,01.01.2021 01:00 - 01.01.2021 01:15,13.71,13.71,Final,17,NaN,Surplus,Intermediate


In [619]:
df_imbalance_de.loc[df_imbalance_de.Situation == "Deficit", 'Situation'] = int(-1)
df_imbalance_de.loc[df_imbalance_de.Situation == "Surplus", 'Situation'] = int(1)
df_imbalance_de.loc[df_imbalance_de.Situation == "Balanced", 'Situation'] = int(0)

df_imbalance_de["Situation"] = df_imbalance_de["Situation"].astype('int64')


#pd.to_numeric(df_imbalance_de.Situation)

#df_imbalance_de




In [620]:
df_imbalance_de.dtypes

Imbalancesettlementperiod(UTC)      object
plusImbalancePriceEURMWhMBADELU    float64
ImbalancePriceEURMWhMBADELU        float64
Status                              object
TotalImbalanceMWhMBADELU             int64
Difference                         float64
Situation                            int64
Status.1                            object
dtype: object

In [621]:
pd.to_numeric(df_imbalance_de.TotalImbalanceMWhMBADELU)

dt_start_utc
2021-01-01 00:00:00    52
2021-01-01 00:15:00     7
2021-01-01 00:30:00    13
2021-01-01 00:45:00    38
2021-01-01 01:00:00    17
                       ..
2021-06-30 20:45:00    57
2021-06-30 21:00:00     4
2021-06-30 21:15:00    62
2021-06-30 21:30:00    11
2021-06-30 21:45:00    44
Name: TotalImbalanceMWhMBADELU, Length: 17368, dtype: int64

In [622]:
df_imbalance_de.Situation.unique()

array([-1,  1,  0])

In [623]:
pd.to_numeric(df_imbalance_de.Situation)

dt_start_utc
2021-01-01 00:00:00   -1
2021-01-01 00:15:00   -1
2021-01-01 00:30:00   -1
2021-01-01 00:45:00    1
2021-01-01 01:00:00    1
                      ..
2021-06-30 20:45:00   -1
2021-06-30 21:00:00    1
2021-06-30 21:15:00   -1
2021-06-30 21:30:00   -1
2021-06-30 21:45:00    1
Name: Situation, Length: 17368, dtype: int64

In [624]:
df_imbalance_de.eval("rz_saldo_mwh = Situation * TotalImbalanceMWhMBADELU", inplace=True)

In [625]:
df_imbalance_de.eval("temp = plusImbalancePriceEURMWhMBADELU - ImbalancePriceEURMWhMBADELU", inplace=True)

In [626]:
df_imbalance_de["temp"].nunique()

1

In [627]:
df_imbalance_de.columns

Index(['Imbalancesettlementperiod(UTC)', 'plusImbalancePriceEURMWhMBADELU',
       'ImbalancePriceEURMWhMBADELU', 'Status', 'TotalImbalanceMWhMBADELU',
       'Difference', 'Situation', 'Status.1', 'rz_saldo_mwh', 'temp'],
      dtype='object')

In [628]:
df_imbalance_de.drop(['Imbalancesettlementperiod(UTC)', 'Status',
                      "ImbalancePriceEURMWhMBADELU",
                      "TotalImbalanceMWhMBADELU", "Difference", 
                      "Situation", "Status.1", "temp"], axis=1, inplace=True)



In [629]:
df_imbalance_de.head()

,plusImbalancePriceEURMWhMBADELU,rz_saldo_mwh
dt_start_utc,,
2021-01-01 00:00:00,96.09,-52
2021-01-01 00:15:00,94.62,-7
2021-01-01 00:30:00,92.21,-13
2021-01-01 00:45:00,27.64,38
2021-01-01 01:00:00,13.71,17


In [630]:


df_imbalance_de.rename(columns={"plusImbalancePriceEURMWhMBADELU": "rebap_eur_mwh"
                              },inplace=True)

df_imbalance_de

,rebap_eur_mwh,rz_saldo_mwh
dt_start_utc,,
2021-01-01 00:00:00,96.09,-52
2021-01-01 00:15:00,94.62,-7
2021-01-01 00:30:00,92.21,-13
2021-01-01 00:45:00,27.64,38
2021-01-01 01:00:00,13.71,17
...,...,...
2021-06-30 20:45:00,119.06,-57
2021-06-30 21:00:00,3.10,4
2021-06-30 21:15:00,125.68,-62


## First exploration

We check whether the import worked as expected.

We want to unify the timezones in all our data set to UTC. The origin timezone is CET and we just need substract one our of the datetime column.

The next step is to transfer columns with an numerical character to an numerical type and rename them to a shorter name.

To keep the dataframe lean, we simplfiy "rzpuls" and "rzminus" to a single "rz_saldo" column and drop both from the dataframe

We can't indentify any duplicates. There's not need to take action. Now we can set the date column to index.

In [631]:
#df_imbalance_de = df_imbalance_de.copy()
print (df_imbalance_de.index.min())
print (df_imbalance_de.index.max())
print (df_imbalance_de.index.is_unique)
#df_imbalance_de.to_csv("test_df_imbalance_de.csv")

2021-01-01 00:00:00
2021-06-30 21:45:00
True


In [632]:
#df_imbalance_de.duplicated().sum()

In [633]:
count_inf = np.isinf(df_imbalance_de).values.sum()
count_nan = df_imbalance_de.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 0 missing values.


In [634]:
df_imbalance_de.shape

(17368, 2)

In [635]:
df_imbalance_de.tail()

,rebap_eur_mwh,rz_saldo_mwh
dt_start_utc,,
2021-06-30 20:45:00,119.06,-57
2021-06-30 21:00:00,3.10,4
2021-06-30 21:15:00,125.68,-62
2021-06-30 21:30:00,100.92,-11
2021-06-30 21:45:00,-19.88,44


## Implement Real Life Time Delays to Data Frame

In real life, the balancing energie value is published with an delay of 30 mins. We remodeling this situation by shifting this column by two rows, which is equivalent to 30 mins.

In [636]:
df_imbalance_de["rz_saldo_mwh"] = df_imbalance_de["rz_saldo_mwh"].shift(periods=2)

In [637]:
#df_imbalance_de.resample(resample_size).agg({'rebap_eur_mwh': np.interp, 'rz_saldo_mwh': np.mean})

In [638]:
df_rz = df_imbalance_de.drop('rebap_eur_mwh', axis=1)

In [639]:
#df_rz.tail()

In [640]:
count_inf = np.isinf(df_imbalance_de).values.sum()
count_nan = df_imbalance_de.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 2 missing values.


In [641]:
#df_rz.fillna(0)

In [642]:
df_rz["rz_saldo_mwh"].duplicated().sum()

16591

In [643]:
df_rz= df_rz[~df_rz.index.duplicated(keep='first')]

In [644]:
df_rz.shape

(17368, 1)

In [645]:
#df_rz = df_rz.resample(resample_size).interpolate(method='polynomial', order=2)

In [646]:
df_rz.head()

,rz_saldo_mwh
dt_start_utc,
2021-01-01 00:00:00,NaN
2021-01-01 00:15:00,NaN
2021-01-01 00:30:00,-52.0
2021-01-01 00:45:00,-7.0
2021-01-01 01:00:00,-13.0


In [647]:
df_imbalance_de.drop('rz_saldo_mwh', axis=1, inplace=True)

In [648]:
df_imbalance_de.tail()

,rebap_eur_mwh
dt_start_utc,
2021-06-30 20:45:00,119.06
2021-06-30 21:00:00,3.10
2021-06-30 21:15:00,125.68
2021-06-30 21:30:00,100.92
2021-06-30 21:45:00,-19.88


In [649]:
#df_imbalance_de = df_imbalance_de[~df_imbalance_de.index.duplicated(keep='first')]

In [650]:
#df_imbalance_de = df_imbalance_de.resample(resample_size).ffill();

In [651]:
df_imbalance_de.tail(15)

,rebap_eur_mwh
dt_start_utc,
2021-06-30 18:15:00,134.91
2021-06-30 18:30:00,137.10
2021-06-30 18:45:00,136.11
2021-06-30 19:00:00,169.18
2021-06-30 19:15:00,172.76
2021-06-30 19:30:00,117.90
2021-06-30 19:45:00,135.15
2021-06-30 20:00:00,163.27
2021-06-30 20:15:00,123.92


In [652]:
df_imbalance_de.to_csv("check_df_imbalance_de.csv")

In [653]:
df_imbalance_de = df_imbalance_de.merge(df_rz, left_index=True, right_index=True)

In [654]:
print("The Data Frame has",df_imbalance_de.isnull().sum().sum(),"missing values.")

The Data Frame has 2 missing values.


In [655]:
#df_imbalance_de.fillna(df_imbalance_de.median(), inplace=True)

In [656]:
df_imbalance_de.head(10)

,rebap_eur_mwh,rz_saldo_mwh
dt_start_utc,,
2021-01-01 00:00:00,96.09,NaN
2021-01-01 00:15:00,94.62,NaN
2021-01-01 00:30:00,92.21,-52.0
2021-01-01 00:45:00,27.64,-7.0
2021-01-01 01:00:00,13.71,-13.0
2021-01-01 01:15:00,11.99,38.0
2021-01-01 01:30:00,4.31,17.0
2021-01-01 01:45:00,-3.01,40.0
2021-01-01 02:00:00,2.22,64.0


In [657]:
print("The data frame has",df_imbalance_de.isnull().sum().sum(),"missing values.")

The data frame has 2 missing values.


In [658]:
df_imbalance_de.to_pickle('data/pickle/df_imbalance_de.pickle')

## 1.1 Import Belgium

In [659]:
df = pd.read_csv("data/entsoe/Imbalance_Belgien.csv")

In [660]:
df[['dt_start_utc','Last']] = df.dt_start_utc.str.split("-", expand=True)

In [661]:
cleaningcolumns(df)

/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('|','')
/var/folders/x_/r

In [662]:
df.loc[df.situation == "Deficit", 'situation'] = int(-1)
df.loc[df.situation == "Surplus", 'situation'] = int(1)
df.loc[df.situation == "Balanced", 'situation'] = int(0)

df["situation"] = df["situation"].astype('int64')

In [663]:
df.head()

,dt_start_utc,plusimbalancepriceeurmwhscabe,imbalancepriceeurmwhscabe,status,totalimbalancemwhscabe,difference,situation,status.1,last
0,01.01.2021 00:00,64.84,64.84,Final,16,-3,-1,Final,01.01.2021 00:15
1,01.01.2021 00:15,13.96,13.96,Final,38,2,1,Final,01.01.2021 00:30
2,01.01.2021 00:30,12.40,12.40,Final,46,1,1,Final,01.01.2021 00:45
3,01.01.2021 00:45,7.70,7.70,Final,65,14,1,Final,01.01.2021 01:00
4,01.01.2021 01:00,64.25,64.25,Final,3,-9,-1,Final,01.01.2021 01:15


In [664]:
df.eval("rz_saldo_mwh_BE = situation * totalimbalancemwhscabe", inplace=True)


In [665]:
df.drop(["imbalancepriceeurmwhscabe", "totalimbalancemwhscabe","last", "status", "status.1", "situation", "difference"], axis=1, inplace=True)
df.rename(columns={"plusimbalancepriceeurmwhscabe": "rebap_eur_mwh_BE"},inplace=True)

In [666]:
df

,dt_start_utc,rebap_eur_mwh_BE,rz_saldo_mwh_BE
0,01.01.2021 00:00,64.84,-16
1,01.01.2021 00:15,13.96,38
2,01.01.2021 00:30,12.40,46
3,01.01.2021 00:45,7.70,65
4,01.01.2021 01:00,64.25,-3
...,...,...,...
20109,29.07.2021 11:15,-53.36,42
20110,29.07.2021 11:30,-106.41,78
20111,29.07.2021 11:45,-123.69,74
20112,29.07.2021 12:00,-108.10,44


In [667]:
df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"], dayfirst=True)

In [668]:
df.set_index('dt_start_utc', inplace=True)

In [669]:
df.head()

,rebap_eur_mwh_BE,rz_saldo_mwh_BE
dt_start_utc,,
2021-01-01 00:00:00,64.84,-16
2021-01-01 00:15:00,13.96,38
2021-01-01 00:30:00,12.40,46
2021-01-01 00:45:00,7.70,65
2021-01-01 01:00:00,64.25,-3


In [670]:
df = df.shift(periods=8)

In [671]:
#df.loc[df["+ Imbalance Price [EUR/MWh] - SCA|BE"] == "-", "+ Imbalance Price [EUR/MWh] - SCA|AT"] = None

In [672]:
#df["+ Imbalance Price [EUR/MWh] - SCA|AT"].astype(float);

In [673]:
#df = df.resample(resample_size).interpolate(method='polynomial', order=2)

In [674]:
count_inf = np.isinf(df_imbalance_de).values.sum()
count_nan = df_imbalance_de.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 2 missing values.


In [675]:
df.to_csv("df_be.csv")

In [676]:
df.head()

,rebap_eur_mwh_BE,rz_saldo_mwh_BE
dt_start_utc,,
2021-01-01 00:00:00,NaN,NaN
2021-01-01 00:15:00,NaN,NaN
2021-01-01 00:30:00,NaN,NaN
2021-01-01 00:45:00,NaN,NaN
2021-01-01 01:00:00,NaN,NaN


In [677]:
#df.fillna(df.median(), inplace=True)

In [678]:
#df.fillna(0, inplace=True)

In [679]:
df_be = df.copy()

## 1.2 Import Poland Data

In [680]:
df = pd.read_csv("data/entsoe/Imbalance_MB__Poland.csv")

In [681]:
df[['dt_start_utc','Last']] = df.dt_start_utc.str.split("-", expand=True)

In [682]:
cleaningcolumns(df)

/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('|','')
/var/folders/x_/r

In [683]:
df.head()

,dt_start_utc,imbalanceprice_pln_mwh_mba_pl,imbalancepriceplnmwhmbapl,status,totalimbalancemwhmbapl,difference,situation,status.1,last
0,01.01.2021 00:00,191.33,191.33,Final,669,36,Surplus,Final,01.01.2021 01:00
1,01.01.2021 01:00,191.33,191.33,Final,521,15,Surplus,Final,01.01.2021 02:00
2,01.01.2021 02:00,191.33,191.33,Final,253,69,Surplus,Final,01.01.2021 03:00
3,01.01.2021 03:00,191.33,191.33,Final,130,47,Surplus,Final,01.01.2021 04:00
4,01.01.2021 04:00,191.33,191.33,Final,35,-85,Surplus,Final,01.01.2021 05:00


In [684]:
df.loc[df.situation == "Deficit", 'situation'] = int(-1)
df.loc[df.situation == "Surplus", 'situation'] = int(1)
df.loc[df.situation == "Balanced", 'situation'] = int(0)

df["situation"] = df["situation"].astype('int64')

df.eval("rz_saldo_mwh_PL = situation * totalimbalancemwhmbapl", inplace=True)
df.drop(["imbalancepriceplnmwhmbapl", "status","totalimbalancemwhmbapl", "status.1", "situation", "difference", "last"], axis=1, inplace=True)
df.rename(columns={"imbalanceprice_pln_mwh_mba_pl": "rebap_eur_mwh_PL"},inplace=True)

In [685]:
df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"], dayfirst=True)

In [686]:
df.set_index('dt_start_utc', inplace=True)

In [687]:
df.head(10)

,rebap_eur_mwh_PL,rz_saldo_mwh_PL
dt_start_utc,,
2021-01-01 00:00:00,191.33,669
2021-01-01 01:00:00,191.33,521
2021-01-01 02:00:00,191.33,253
2021-01-01 03:00:00,191.33,130
2021-01-01 04:00:00,191.33,35
2021-01-01 05:00:00,191.33,-317
2021-01-01 06:00:00,191.33,-445
2021-01-01 07:00:00,191.33,-377
2021-01-01 08:00:00,191.33,141


In [688]:
df = df.resample(resample_size).ffill()

In [689]:
df.head()

,rebap_eur_mwh_PL,rz_saldo_mwh_PL
dt_start_utc,,
2021-01-01 00:00:00,191.33,669
2021-01-01 00:15:00,191.33,669
2021-01-01 00:30:00,191.33,669
2021-01-01 00:45:00,191.33,669
2021-01-01 01:00:00,191.33,521


The data are published one hour delayde

In [690]:
df = df.shift(periods=4)

In [691]:
df.head()

,rebap_eur_mwh_PL,rz_saldo_mwh_PL
dt_start_utc,,
2021-01-01 00:00:00,NaN,NaN
2021-01-01 00:15:00,NaN,NaN
2021-01-01 00:30:00,NaN,NaN
2021-01-01 00:45:00,NaN,NaN
2021-01-01 01:00:00,191.33,669.0


In [692]:
#df = df.select_dtypes(include=['float64']).astype(np.float32)
#df = df.astype(np.float32)

In [693]:
#df.loc[df["+ Imbalance Price [EUR/MWh] - SCA|PL"] == "-", "+ Imbalance Price [EUR/MWh] - SCA|AT"] = None

In [694]:
count = np.isinf(df).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 8 missing values.


In [695]:
df.to_csv("df_pl.csv")

In [696]:
df.head()

,rebap_eur_mwh_PL,rz_saldo_mwh_PL
dt_start_utc,,
2021-01-01 00:00:00,NaN,NaN
2021-01-01 00:15:00,NaN,NaN
2021-01-01 00:30:00,NaN,NaN
2021-01-01 00:45:00,NaN,NaN
2021-01-01 01:00:00,191.33,669.0


In [697]:
#df.fillna(df.median(), inplace=True)

In [698]:
#df.fillna(0, inplace=True)

In [699]:
df_pl = df.copy()

In [700]:
#df["+ Imbalance Price [EUR/MWh] - SCA|AT"].astype(float);

## 1.4 Import Austrian Data

In [701]:
df = pd.read_csv("data/entsoe/AUS_Imba.csv")

In [702]:
df[['dt_start_utc','Last']] = df.dt_start_utc.str.split("-", expand=True)

In [703]:
cleaningcolumns(df)

/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('|','')
/var/folders/x_/r

In [704]:
df.head()

,dt_start_utc,plusimbalancepriceeurmwhscaat,imbalancepriceeurmwhscaat,status,totalimbalancemwhscaat,difference,situation,status.1,last
0,01.01.2021 00:00,85.42,85.42,Estimated,34,NaN,NaN,Final,01.01.2021 00:15
1,01.01.2021 00:15,84.96,84.96,Estimated,40,NaN,NaN,Final,01.01.2021 00:30
2,01.01.2021 00:30,80.71,80.71,Estimated,24,NaN,NaN,Final,01.01.2021 00:45
3,01.01.2021 00:45,70.53,70.53,Estimated,14,NaN,NaN,Final,01.01.2021 01:00
4,01.01.2021 01:00,77.71,77.71,Estimated,32,NaN,NaN,Final,01.01.2021 01:15


In [705]:
df.situation = df.situation.fillna(value=1)
df.loc[df.situation == "Deficit", 'situation'] = int(-1)
df.loc[df.situation == "Surplus", 'situation'] = int(1)
df.loc[df.situation == "Balanced", 'situation'] = int(0)


df["situation"] = df["situation"].astype('int64')

df.eval("rz_saldo_mwh_AUS = situation * totalimbalancemwhscaat", inplace=True)
df.drop(["plusimbalancepriceeurmwhscaat", "status","totalimbalancemwhscaat", "status.1", "situation", "difference", "last"], axis=1, inplace=True)
df.rename(columns={"imbalancepriceeurmwhscaat": "rebap_eur_mwh_AUS"},inplace=True)

In [706]:
df

,dt_start_utc,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS
0,01.01.2021 00:00,85.42,34
1,01.01.2021 00:15,84.96,40
2,01.01.2021 00:30,80.71,24
3,01.01.2021 00:45,70.53,14
4,01.01.2021 01:00,77.71,32
...,...,...,...
20084,29.07.2021 05:00,13.48,8
20085,29.07.2021 05:15,86.98,-28
20086,29.07.2021 05:30,100.00,-45
20087,29.07.2021 05:45,86.98,-30


In [707]:
#df['dt_start_utc'].to_datetime(format='%d.%m.%Y %H:%M', errors='ignore')

#df['dt_start_utc'] = pd.to_datetime(df["dt_start_utc"],format='%d.%m.%Y %H:%M' )

#df['DataFrame Column'] = pd.to_datetime(df['DataFrame Column'], format=specify your format)
#df_aus['dt_start_utc'] = pd.to_datetime(df_aus['dt_start_utc'], format='%d.%m.%Y %H:%M')

#df['dt_start_utc'] = df['dt_start_utc'].apply(lambda _: datetime.strptime(_,format='%d.%m.%Y %H:%M'))

In [708]:
df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"], dayfirst=True)

In [709]:
df

,dt_start_utc,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS
0,2021-01-01 00:00:00,85.42,34
1,2021-01-01 00:15:00,84.96,40
2,2021-01-01 00:30:00,80.71,24
3,2021-01-01 00:45:00,70.53,14
4,2021-01-01 01:00:00,77.71,32
...,...,...,...
20084,2021-07-29 05:00:00,13.48,8
20085,2021-07-29 05:15:00,86.98,-28
20086,2021-07-29 05:30:00,100.00,-45
20087,2021-07-29 05:45:00,86.98,-30


In [710]:
#df_aus.eva("Im_sum = + Imbalance Price [EUR/MWh] - SCA|AT")

In [711]:
df.set_index('dt_start_utc', inplace=True)

In [712]:
df.to_csv("df_aus.csv")

In [713]:
#df_aus.loc[df_aus["+ Imbalance Price [EUR/MWh] - SCA|AT"] == "-", "+ Imbalance Price [EUR/MWh] - SCA|AT"] = None

In [714]:
#df_aus["+ Imbalance Price [EUR/MWh] - SCA|AT"].astype(float);

In [715]:
df.head()

,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS
dt_start_utc,,
2021-01-01 00:00:00,85.42,34
2021-01-01 00:15:00,84.96,40
2021-01-01 00:30:00,80.71,24
2021-01-01 00:45:00,70.53,14
2021-01-01 01:00:00,77.71,32


In [716]:
df = df.shift(periods=10)

In [717]:
#df_aus = df_aus.resample(resample_size).interpolate(method='polynomial', order=2)

In [718]:
#df_aus.fillna(df_aus.median(), inplace=True)

In [719]:
df.to_csv("df_aus.csv")

In [720]:
#df_aus.fillna(0, inplace=True)

In [721]:
df_aus = df.copy()

## 1.5 Import Danemark

In [722]:
df = pd.read_csv("data/entsoe/Imbalance_MB__DK.csv")

In [723]:
df[['dt_start_utc','Last']] =df.dt_start_utc.str.split("-", expand=True)

In [724]:
cleaningcolumns(df)

/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('|','')
/var/folders/x_/r

In [725]:
df.drop(["last", "status", "status.1", "situation", "difference"], axis=1, inplace=True)

In [726]:
df.head()

,dt_start_utc,generationplusimbalancepriceeurmwhmbadk2,generationimbalancepriceeurmwhmbadk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2
0,01.01.2021 00:00,48.19,48.19,48.19,141
1,01.01.2021 01:00,44.68,44.68,44.68,148
2,01.01.2021 02:00,42.92,42.92,42.92,156
3,01.01.2021 03:00,40.39,40.39,40.39,196
4,01.01.2021 04:00,40.20,40.20,40.20,216


In [727]:
df.to_csv("df_dk.csv")

In [728]:
df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"], dayfirst=True)

In [729]:
df.set_index('dt_start_utc', inplace=True)

In [730]:
df = df.resample(resample_size).ffill();

We need to forward the value for 2.5 hours.

In [731]:
df = df.shift(periods=10)

In [732]:
df.head()

,generationplusimbalancepriceeurmwhmbadk2,generationimbalancepriceeurmwhmbadk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2
dt_start_utc,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,NaN,NaN,NaN,NaN


In [733]:


#df = df.resample(resample_size).interpolate(method='polynomial', order=2)

In [734]:
#df.loc[df["+ Imbalance Price [EUR/MWh] - SCA|PL"] == "-", "+ Imbalance Price [EUR/MWh] - SCA|AT"] = None

In [735]:
#df["+ Imbalance Price [EUR/MWh] - SCA|AT"].astype(float);

In [736]:
#df.fillna(df.median(), inplace=True)

In [737]:
df.to_csv("df_dk.csv")

In [738]:
#df.fillna(0, inplace=True)

In [739]:
df_dk = df.copy()

## 1.6 Import Danemark DK1

In [740]:
df = pd.read_csv("data/entsoe/Imbalance_MB__DK_DK1.csv")

In [741]:
df[['dt_start_utc','Last']] =df.dt_start_utc.str.split("-", expand=True)

In [742]:
cleaningcolumns(df)

/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('(', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(')', '')
/var/folders/x_/rjggxhdx2j5ch0bl5gfg01480000gn/T/ipykernel_649/2771464415.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('|','')
/var/folders/x_/r

In [743]:
df.drop(["last", "status", "status.1", "situation", "difference"], axis=1, inplace=True)

In [744]:
df["dt_start_utc"] = pd.to_datetime(df["dt_start_utc"], dayfirst=True)

In [745]:
df.set_index('dt_start_utc', inplace=True)

In [746]:
df.head()

,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,
2021-01-01 00:00:00,48.19,48.19,48.19,75
2021-01-01 01:00:00,44.68,44.68,44.68,86
2021-01-01 02:00:00,42.92,42.92,42.92,30
2021-01-01 03:00:00,40.39,40.39,40.39,84
2021-01-01 04:00:00,40.20,40.20,40.20,30


In [747]:
df = df.resample(resample_size).ffill();

In [748]:
df.head()

,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,
2021-01-01 00:00:00,48.19,48.19,48.19,75
2021-01-01 00:15:00,48.19,48.19,48.19,75
2021-01-01 00:30:00,48.19,48.19,48.19,75
2021-01-01 00:45:00,48.19,48.19,48.19,75
2021-01-01 01:00:00,44.68,44.68,44.68,86


We need to shift 2.5hours forward.

In [749]:
df = df.shift(periods=10)

In [750]:
#df.loc[df["+ Imbalance Price [EUR/MWh] - SCA|PL"] == "-", "+ Imbalance Price [EUR/MWh] - SCA|AT"] = None

In [751]:
#df.fillna(df.median(), inplace=True)

In [752]:
df.head()

,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,NaN,NaN,NaN,NaN


In [753]:
#df.fillna(0, inplace=True)

In [754]:
df.to_csv("df_dk_1.csv")

In [755]:
#df["+ Imbalance Price [EUR/MWh] - SCA|AT"].astype(float);

In [756]:
df_dk_1 = df.copy()

## Data Merge 

In [757]:
dfs = [df_imbalance_de, df_be, df_pl, df_aus, df_dk, df_dk_1]

df_merged_2 = reduce(lambda  left,right: pd.merge(left, right, on=['dt_start_utc'],
                                            how='inner'), dfs)

In [758]:
#df_merged_2 = df_merged_2.drop(df_merged_2.filter(regex='Situation').columns, axis=1)
#df_merged_2 = df_merged_2.drop(df_merged_2.filter(regex='Status').columns, axis=1)
#df_merged_2.loc[:,~df_merged_2.columns.str.startswith('Situation')]

In [759]:
df_merged_2

,rebap_eur_mwh,rz_saldo_mwh,rebap_eur_mwh_BE,rz_saldo_mwh_BE,rebap_eur_mwh_PL,rz_saldo_mwh_PL,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS,generationplusimbalancepriceeurmwhmbadk2,generationimbalancepriceeurmwhmbadk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,96.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,94.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,92.21,-52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,27.64,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,13.71,-13.0,NaN,NaN,191.33,669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30 20:45:00,119.06,-79.0,104.51,-14.0,351.15,712.0,-47.04,-102.0,98.97,268.93,268.93,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:00:00,3.10,-91.0,116.67,-50.0,358.31,343.0,-97.86,-87.0,98.97,268.93,268.93,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:15:00,125.68,-57.0,119.98,-49.0,358.31,343.0,-199.94,-71.0,98.97,268.93,268.93,220.0,98.97,268.93,268.93,100.0


In [760]:
df_merged_2.head()

,rebap_eur_mwh,rz_saldo_mwh,rebap_eur_mwh_BE,rz_saldo_mwh_BE,rebap_eur_mwh_PL,rz_saldo_mwh_PL,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS,generationplusimbalancepriceeurmwhmbadk2,generationimbalancepriceeurmwhmbadk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,96.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:15:00,94.62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:30:00,92.21,-52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:45:00,27.64,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00,13.71,-13.0,NaN,NaN,191.33,669.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [761]:
df_merged_2.shape

(17368, 16)

In [762]:
df_merged_2.tail()

,rebap_eur_mwh,rz_saldo_mwh,rebap_eur_mwh_BE,rz_saldo_mwh_BE,rebap_eur_mwh_PL,rz_saldo_mwh_PL,rebap_eur_mwh_AUS,rz_saldo_mwh_AUS,generationplusimbalancepriceeurmwhmbadk2,generationimbalancepriceeurmwhmbadk2,priceforconsumptioneurmwhmbadk2,totalimbalancemwhmbadk2,generationplusimbalancepriceeurmwhmbadk1,generationimbalancepriceeurmwhmbadk1,priceforconsumptioneurmwhmbadk1,totalimbalancemwhmbadk1
dt_start_utc,,,,,,,,,,,,,,,,
2021-06-30 20:45:00,119.06,-79.0,104.51,-14.0,351.15,712.0,-47.04,-102.0,98.97,268.93,268.93,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:00:00,3.10,-91.0,116.67,-50.0,358.31,343.0,-97.86,-87.0,98.97,268.93,268.93,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:15:00,125.68,-57.0,119.98,-49.0,358.31,343.0,-199.94,-71.0,98.97,268.93,268.93,220.0,98.97,268.93,268.93,100.0
2021-06-30 21:30:00,100.92,4.0,104.51,-9.0,358.31,343.0,171.06,56.0,46.00,97.02,46.00,97.0,46.00,97.02,46.00,39.0
2021-06-30 21:45:00,-19.88,-62.0,104.51,-15.0,358.31,343.0,175.22,86.0,46.00,97.02,46.00,97.0,46.00,97.02,46.00,39.0


In [763]:
count_inf = np.isinf(df_merged_2).values.sum()
count_nan = df_merged_2.isnull().sum().sum()
print("The data frame contains " + str(count_inf) + " infinite values and " + str(count_nan) + " missing values.")

The data frame contains 0 infinite values and 126 missing values.


In [764]:
df_merged_2.to_csv("df_merged_2_test.csv")

## Save Data Set

In [765]:
count = np.isinf(df_merged_2).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df_merged_2.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 126 missing values.


In [766]:
df_merged_2.to_pickle('data/pickle/df_merged_2.pickle')